Kelompok 11
<br>
C14210005 - Joshua Briantama Hanjaya
<br>
C14210177 - Alfred Wisana
<br>
C14210185 - Frederika Handoyo

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.metrics import confusion_matrix, roc_auc_score, mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

No 1: Introduction to Dataset

In [ ]:
df = pd.read_csv(url)

In [ ]:
# encoding preprocessing cleaning
df['yesno'] = df['yesno'].replace('y',1)
df['yesno'] = df['yesno'].replace('n',0)
print(df.isnull().sum())
df.head()

In [ ]:
# correlation matrix
df.corr()

In [ ]:
df.describe()

In [ ]:
df_copy = df.copy(deep=True)

In [ ]:
scaler = MinMaxScaler()
df_copy['crl.tot'] = scaler.fit_transform(df_copy[['crl.tot']])
df_copy

In [ ]:
X = df_copy.drop('yesno', axis=1)
y = df_copy.yesno
#data dipisah menjadi training dan testing
X_temp, X_test, y_temp, y_test = train_test_split(X,y,test_size=0.3,random_state=123)
#data dipisah menjadi trainging dan validation
X_train, X_val, y_train, y_val = train_test_split(X_temp,y_temp,test_size=0.3,random_state=123)

NO. 3: Machine Learning Algorithm Using Logistic Regression

In [ ]:
#mencari p value
model_sm = sm.Logit(y_train,X_train)
result = model_sm.fit()
result.summary()

In [ ]:
#Akurasi test data
logistic_regression = LogisticRegression(solver='lbfgs',max_iter=200, random_state=1234)
logistic_regression.fit(X_train,y_train)

y_test_pred = logistic_regression.predict(X_test)
acc_score=accuracy_score(y_test,y_test_pred)
print('accuracy score = %0.2f'%acc_score)

In [ ]:
auc_score = roc_auc_score(y_test,y_test_pred)
print('AUC Score: %0.2f'%auc_score)

Logistic Regression Graphic

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(2,3,1)
sns.regplot(x=df['crl.tot'], y=df['yesno'], logistic=True, ci=None)
plt.subplot(2,3,2)
sns.regplot(x=df['dollar'], y=df['yesno'], logistic=True, ci=None)
plt.subplot(2,3,3)
sns.regplot(x=df['bang'], y=df['yesno'], logistic=True, ci=None)
plt.subplot(2,3,4)
sns.regplot(x=df['make'], y=df['yesno'], logistic=True, ci=None)
plt.subplot(2,3,5)
sns.regplot(x=df['money'], y=df['yesno'], logistic=True, ci=None)
plt.subplot(2,3,6)
sns.regplot(x=df['n000'], y=df['yesno'], logistic=True, ci=None)

plt.tight_layout()
plt.show()

CONFOUNDING

In [ ]:
X_crltot_bang = X_train[['bang','make']]
model_confounding = sm.Logit(y_train,X_crltot_bang)
res = model_confounding.fit()
res.summary()

In [ ]:
lr = LogisticRegression()
lr.fit(X_train[['make']],y_train)
y_cf_pred = lr.predict(X_val[['make']])
roc_auc_score(y_val,y_cf_pred)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train[['bang']],y_train)
y_cf_pred = lr.predict(X_val[['bang']])
roc_auc_score(y_val,y_cf_pred)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train[['bang','make']],y_train)
y_cf_pred = lr.predict(X_val[['bang','make']])
roc_auc_score(y_val,y_cf_pred)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train.drop('make',axis=1),y_train)
y_cf_pred = lr.predict(X_val.drop('make',axis=1))
roc_auc_score(y_val,y_cf_pred)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train.drop('crl.tot',axis=1),y_train)
y_cf_pred = lr.predict(X_val.drop('crl.tot',axis=1))
roc_auc_score(y_val,y_cf_pred)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train.drop('bang',axis=1),y_train)
y_cf_pred = lr.predict(X_val.drop('bang',axis=1))
roc_auc_score(y_val,y_cf_pred)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train.drop('money',axis=1),y_train)
y_cf_pred = lr.predict(X_val.drop('money',axis=1))
roc_auc_score(y_val,y_cf_pred)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train.drop('n000',axis=1),y_train)
y_cf_pred = lr.predict(X_val.drop('n000',axis=1))
roc_auc_score(y_val,y_cf_pred)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train.drop('dollar',axis=1),y_train)
y_cf_pred = lr.predict(X_val.drop('dollar',axis=1))
roc_auc_score(y_val,y_cf_pred)

CONFUSION MATRIX

In [ ]:
cf = confusion_matrix(y_test, y_test_pred, normalize='true')
plt.figure(figsize=(3,2))
sns.heatmap(cf, annot=True)
plt.xlabel('Prediction')
plt.ylabel('Target')
plt.title('{} Confusion Matrix'.format('validation'));

ROC AUC SCORE

In [ ]:
y_val_pred = logistic_regression.predict(X_val)
y_train_pred = logistic_regression.predict(X_train)

In [ ]:
roc_val = roc_auc_score(y_val,y_val_pred)
print('roc_auc_score = %0.2f'%roc_val)
roc_val = round(roc_val,2)

In [ ]:
# Training Data
roc_train = roc_auc_score(y_train,y_train_pred)
print('roc_auc_score = %0.2f'%roc_train)
roc_train = round(roc_train,2)

In [ ]:
if roc_val < roc_train:
    print("The Model is overfit")
elif roc_val > roc_train:
    print("The Model may be underfit")
else:
    print("The Model is neither overfit nor underfit")

NO 4. Resampling Method

Resampling menggunakan validation set.

In [ ]:
mse = mean_squared_error(y_val,y_val_pred)
print("MSE: %0.2f"%mse)

In [ ]:
#Validation Data
y_val_pred = logistic_regression.predict(X_val)
acc_score_val=accuracy_score(y_val,y_val_pred)
print('accuracy score = %0.2f'%acc_score)

In [ ]:
#Train Data
y_train_pred = logistic_regression.predict(X_train)
acc_score_train=accuracy_score(y_train,y_train_pred)
print('accuracy score = %0.2f'%acc_score)

In [ ]:
# k-fold cross validation
score = cross_val_score(logistic_regression,X_train,y_train,scoring=None,cv=10)
score_mean=score.mean()
print('Score Mean: %0.2f'%score_mean)

KFOLD VALIDATION

In [ ]:
#evaluasi menggunakan kfold cross validation
kf = KFold(n_splits=10)
# pembagian ke dalam 10 fold untuk kfold cross validation
mse_scores = []
fold = 1
for train_index, val_index in kf.split(X_temp):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    logistic_regression.fit(X_train,y_train)
    y_pred = logistic_regression.predict(X_val)
    mse = mean_squared_error(y_val,y_pred)
    acc_score = accuracy_score(y_val,y_pred)
    mse_scores.append(mse)
    print("="*50)
    print(f"Fold : {fold}")
    print("="*50)
    print("mean squared error %0.2f"%mse)
    print("accuracy score %0.2f"%acc_score)
    print("="*50)
    fold+=1
    
avg_mse = sum(mse_scores)/len(mse_scores)
print("Average MSE Across All Folds:", avg_mse)
print("score:", accuracy_score(y_val,y_pred))

In [ ]:
#testing data
y_test_pred = logistic_regression.predict(X_test)
acc=accuracy_score(y_test,y_test_pred)
print("Accuracy: %0.2f"%acc)

NO. 5: Pemilihan Model dan metode memperbaiki model

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(X,y,test_size=0.3,random_state=123)
#data dipisah menjadi trainging dan validation
X_train, X_val, y_train, y_val = train_test_split(X_temp,y_temp,test_size=0.3,random_state=123)

NAIVE BAYES MODEL

In [ ]:
gaussianNB = GaussianNB()

In [ ]:
gaussianNB.fit(X_train,y_train)
gaussianNB_y_pred_val = gaussianNB.predict(X_val)

In [ ]:
gaussianNB_acc_score = accuracy_score(y_val,gaussianNB_y_pred_val)
print("Gaussian Accuracy Score: %0.2f" %gaussianNB_acc_score)

In [ ]:
gaussianNB_y_pred_test = gaussianNB.predict(X_test)

In [ ]:
gaussianNB_acc_score_test = accuracy_score(y_test,gaussianNB_y_pred_test)
print("Gaussian Accuracy Score: %0.2f" %gaussianNB_acc_score_test)

In [ ]:
cf = confusion_matrix(y_val, gaussianNB_y_pred_val, normalize='true')
plt.figure(figsize=(3,2))
sns.heatmap(cf, annot=True)
plt.xlabel('Prediction')
plt.ylabel('Target')
plt.title('{} Confusion Matrix'.format('validation'));

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis

LDA METHOD

In [ ]:
LDA = LinearDiscriminantAnalysis()

In [ ]:
LDA.fit(X_train,y_train)
LDA_y_pred_val = LDA.predict(X_val)

In [ ]:
LDA_acc_score = accuracy_score(y_val,LDA_y_pred_val)
print('LDA Accuracy Score: %0.2f'%LDA_acc_score)
# LDA_acc_score

In [ ]:
LDA_y_pred_test = LDA.predict(X_test)

In [ ]:
LDA_acc_score_test = accuracy_score(y_test,LDA_y_pred_test)
print("LDA Accuracy Score: %0.2f" %LDA_acc_score_test)

In [ ]:
cf = confusion_matrix(y_val, LDA_y_pred_val, normalize='true')
plt.figure(figsize=(3,2))
sns.heatmap(cf, annot=True)
plt.xlabel('Prediction')
plt.ylabel('Target')
plt.title('{} Confusion Matrix'.format('validation'));

QDA METHOD

In [ ]:
QDA = QuadraticDiscriminantAnalysis()

In [ ]:
QDA.fit(X_train,y_train)
QDA_y_pred_val = QDA.predict(X_val)

In [ ]:
QDA_acc_score = accuracy_score(y_val,QDA_y_pred_val)
print("QDA Accuracy Score: %0.2f"%QDA_acc_score)

In [ ]:
QDA_y_pred_test = QDA.predict(X_test)

In [ ]:
QDA_acc_score_test = accuracy_score(y_test,QDA_y_pred_test)
print("QDA Accuracy Score: %0.2f" %gaussianNB_acc_score_test)

In [ ]:
cf = confusion_matrix(y_val, QDA_y_pred_val, normalize='true')
plt.figure(figsize=(3,2))
sns.heatmap(cf, annot=True)
plt.xlabel('Prediction')
plt.ylabel('Target')
plt.title('{} Confusion Matrix'.format('validation'));